# Generate pressure coefficients

The following steps are performed to obtain all the artifacts for pressure coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data
   * For reference static pressure
   * For body pressure data
   * Slice from timestep range specified
4. Transform pressure data into pressure coefficient
5. Calculate pressure coefficient statistics
6. Merge coefficient statistics with mesh description into VTK
  
  

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager, copy_input_artifacts
import pathlib

path_manager = CpPathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Cp config file

In [2]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)
cfg = post_proc_cfg.pressure_coefficient

post_proc_cfg

CpCaseConfig(pressure_coefficient=CpConfig(timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, U_H_correction_factor=1, statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']))

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read hist series and slice from the time range

In [4]:
from cfdmod.use_cases.pressure.cp_data import filter_pressure_data
import pandas as pd

static_data_path = pathlib.Path(
    "./fixtures/tests/pressure/data/points.static_pressure.data.resampled.h5"
)
body_data_path = pathlib.Path("./fixtures/tests/pressure/data/bodies.galpao.data.resampled.h5")

press_data: pd.DataFrame = pd.read_hdf(static_data_path)  # type: ignore
body_data: pd.DataFrame = pd.read_hdf(body_data_path)  # type: ignore

press_data, body_data = filter_pressure_data(press_data, body_data, cfg.timestep_range)

press_data, body_data

(      point_idx       rho  time_step
 2915          0  1.002515    10000.0
 2915          0  1.002532    10001.0
 2915          0  1.002549    10002.0
 2915          0  1.002566    10003.0
 2915          0  1.002583    10004.0
 ...         ...       ...        ...
 2915          0  1.003020    10096.0
 2915          0  1.003010    10097.0
 2915          0  1.002998    10098.0
 2915          0  1.002987    10099.0
 2915          0  1.002974    10100.0
 
 [101 rows x 3 columns],
       point_idx       rho  time_step
 0             0  1.001819    10000.0
 1             1  1.001838    10000.0
 2             2  1.001808    10000.0
 3             3  1.001851    10000.0
 4             4  1.001835    10000.0
 ...         ...       ...        ...
 2910       2910  1.002633    10100.0
 2911       2911  1.002594    10100.0
 2912       2912  1.002504    10100.0
 2913       2913  1.002828    10100.0
 2914       2914  1.002812    10100.0
 
 [294415 rows x 3 columns])

Transform pressure data into coefficient

In [5]:
from cfdmod.use_cases.pressure.cp_data import transform_to_cp

cp_data = transform_to_cp(
    press_data,
    body_data,
    reference_vel=cfg.U_H,
    ref_press_mode=cfg.reference_pressure,
    correction_factor=cfg.U_H_correction_factor,
)

cp_data

,time_step,point_idx,cp
0,10000.0,0,-0.311587
1,10000.0,1,-0.306326
2,10000.0,2,-0.314535
3,10000.0,3,-0.302966
4,10000.0,4,-0.307277
...,...,...,...
294410,10100.0,2910,-0.094988
294411,10100.0,2911,-0.105542
294412,10100.0,2912,-0.129440
294413,10100.0,2913,-0.043326


Calculate pressure coefficient statistics

In [6]:
from cfdmod.use_cases.pressure.cp_data import calculate_statistics

cp_stats = calculate_statistics(cp_data, statistics_to_apply=cfg.statistics)

cp_stats

,point_idx,cp_avg,cp_min,cp_max,cp_std,cp_skewness,cp_kurtosis
0,0,-0.090791,-0.311587,0.073436,0.112404,-0.289693,-1.070327
1,1,-0.085373,-0.306326,0.078444,0.112650,-0.288853,-1.076454
2,2,-0.092939,-0.314535,0.072865,0.112820,-0.284845,-1.066670
3,3,-0.082793,-0.302966,0.079394,0.112221,-0.294024,-1.079342
4,4,-0.086478,-0.307277,0.077303,0.112585,-0.289346,-1.076362
...,...,...,...,...,...,...,...
2910,2910,-0.259323,-0.437477,-0.094988,0.107410,-0.078239,-1.323681
2911,2911,-0.213272,-0.343250,-0.105542,0.077080,-0.113505,-1.393316
2912,2912,-0.236093,-0.362013,-0.129440,0.076500,-0.115368,-1.420711
2913,2913,-0.228121,-0.474877,-0.043326,0.138243,-0.243726,-1.306376


Combine statistics with mesh and create a VTK object

In [7]:
from cfdmod.api.vtk.write_vtk import write_polydata, create_polydata_for_cell_data

polydata = create_polydata_for_cell_data(data=cp_stats, mesh=mesh.geometry)

# OUTPUT 3: VTK cp_stats
write_polydata(path_manager.vtp_path, polydata)

Copy Input artifacts to output folder

In [8]:
copy_input_artifacts(
    cfg_path=cfg_path,
    mesh_path=mesh_path,
    static_data_path=static_data_path,
    body_data_path=body_data_path,
    path_manager=path_manager,
)